In [2]:
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from simtk import unit, openmm
from openmm import app
import mdtraj as md


In [7]:
from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory, RESTCapableHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
from openmoltools import forcefield_generators
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

import numpy as np
import mdtraj as md
import copy

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS', 'HID', 'HIE', 'HIP']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)

class PointMutationExecutor2(PointMutationExecutor):
   
    def __init__(self,
                 protein_filename,
                 mutation_chain_id,
                 mutation_residue_id,
                 proposed_residue,
                 old_residue=None,
                 phase='complex',
                 use_counterion=True,
                 conduct_endstate_validation=True,
                 ligand_input=None,
                 ligand_index=0,
                 allow_undefined_stereo_sdf=False,
                 extra_sidechain_map=None,
                 demap_CBs=False,
                 solvate=True,
                 water_model='tip3p',
                 ionic_strength=0.15 * unit.molar,
                 forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                 barostat=openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50),
                 forcefield_kwargs={'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus},
                 periodic_forcefield_kwargs={'nonbondedMethod': app.PME},
                 nonperiodic_forcefield_kwargs=None,
                 small_molecule_forcefields='gaff-2.11',
                 complex_box_dimensions=None,
                 apo_box_dimensions=None,
                 flatten_torsions=False,
                 flatten_exceptions=False,
                 rest_radius=0.2,
                 w_scale=0.2,
                 generate_unmodified_hybrid_topology_factory=True,
                 generate_repartitioned_hybrid_topology_factory=False,
                 generate_rest_capable_hybrid_topology_factory=False,
                 **kwargs):
        """
        arguments
            protein_filename : str
                path to protein (to mutate); .pdb
                Note: if there are nonstandard residues, the PDB should contain the standard residue name but the atoms/positions should correspond to the nonstandard residue. E.g. if I want to include HID, the PDB should contain HIS for the residue name, but the atoms should correspond to the atoms present in HID. You can use openmm.app.Modeller.addHydrogens() to generate a PDB like this. The same is true for the ligand_input, if its a PDB. 
                Note: this can be the protein solute only or the solvated protein.
            mutation_chain_id : str
                name of the chain to be mutated
            mutation_residue_id : str
                residue id to change
            proposed_residue : str
                three letter code of the residue to mutate to
            old_residue : str, default None
                name of the old residue, if is a nonstandard amino acid (e.g. LYN, ASH, HID, etc)
                if not specified, the old residue name will be inferred from the input PDB.
            phase : str, default complex
                if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
            conduct_endstate_validation : bool, default True
                whether to conduct an endstate validation of the HybridTopologyFactory. If using the RepartitionedHybridTopologyFactory,
                endstate validation cannot and will not be conducted.
            ligand_input : str, default None
                path to ligand of interest (i.e. small molecule or protein)
                Note: if this is not solvated, it should be the ligand alone (.sdf or .pdb). if solvated, this should be the protein + ligand (.pdb).
            ligand_index : int, default 0
                which ligand to use
            allow_undefined_stereo_sdf : bool, default False
                whether to allow an SDF file to contain undefined stereocenters
            extra_sidechain_map : dict, key: int, value: int, default None
                map of new to old sidechain atom indices to add to the default map (by default, we only map backbone atoms and CBs)
            demap_CBs : bool, default False
                whether to remove CBs from the mapping
            solvate : bool, default True
                whether to solvate the protein/complex
            water_model : str, default 'tip3p'
                solvent model to use for solvation
            ionic_strength : float * unit.molar, default 0.15 * unit.molar
                the total concentration of ions (both positive and negative) to add using Modeller.
                This does not include ions that are added to neutralize the system.
                Note that only monovalent ions are currently supported.
            forcefield_files : list of str, default ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
                forcefield files for proteins and solvent
            barostat : openmm.MonteCarloBarostat, default openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
                barostat to use
            forcefield_kwargs : dict, default {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}
                forcefield kwargs for system parametrization
            periodic_forcefield_kwargs : dict, default {'nonbondedMethod': app.PME}
                periodic forcefield kwargs for system parametrization
            nonperiodic_forcefield_kwargs : dict, default None
                non-periodic forcefield kwargs for system parametrization
            small_molecule_forcefields : str, default 'gaff-2.11'
                the forcefield string for small molecule parametrization
            complex_box_dimensions : Vec3, default None
                define box dimensions of complex phase;
                if None, padding is 1nm
            apo_box_dimensions :  Vec3, default None
                define box dimensions of apo phase phase;
                if None, padding is 1nm
            flatten_torsions : bool, default False
                in the htf, flatten torsions involving unique new atoms at lambda = 0 and unique old atoms are lambda = 1
            flatten_exceptions : bool, default False
                in the htf, flatten exceptions involving unique new atoms at lambda = 0 and unique old atoms at lambda = 1
            generate_unmodified_hybrid_topology_factory : bool, default True
                whether to generate a vanilla HybridTopologyFactory
            generate_rest_capable_hybrid_topology_factory : bool, default False
                whether to generate a RepartitionedHybridTopologyFactory
        TODO : allow argument for spectator ligands besides the 'ligand_file'
        """
        from openeye import oechem

        # First thing to do is load the apo protein to mutate...
        if protein_filename.endswith('pdb'):
            protein_pdb = app.PDBFile(protein_filename)
        elif protein_filename.endswith('cif'):
            protein_pdb = app.PDBxFile(protein_filename)
        else:
            raise Exception("protein_filename file format is not supported. supported formats: .pdb, .cif")
        protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
        protein_n_atoms = protein_md_topology.n_atoms
    
        # Load the ligand, if present
        molecules = []
        if ligand_input:
            if isinstance(ligand_input, str):
                if ligand_input.endswith('.sdf'): # small molecule
                        ligand_mol = createOEMolFromSDF(ligand_input, index=ligand_index, allow_undefined_stereo=allow_undefined_stereo_sdf)
                        molecules.append(Molecule.from_openeye(ligand_mol, allow_undefined_stereo=False))
                        ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_mol),  forcefield_generators.generateTopologyFromOEMol(ligand_mol)
                        ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                        ligand_n_atoms = ligand_md_topology.n_atoms

                elif ligand_input.endswith('pdb'): # protein
                    ligand_pdb = app.PDBFile(ligand_input)
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

                elif ligand_input.endswith('cif'): # protein
                    ligand_pdbfile = open(ligand_input, 'r')
                    ligand_pdb = app.PDBxFile(ligand_pdbfile)
                    ligand_pdbfile.close()
                    ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
                        ligand_pdb.topology)
                    ligand_n_atoms = ligand_md_topology.n_atoms

            elif isinstance(ligand_input, oechem.OEMol): # oemol object
                molecules.append(Molecule.from_openeye(ligand_input, allow_undefined_stereo=False))
                ligand_positions, ligand_topology = extractPositionsFromOEMol(ligand_input),  forcefield_generators.generateTopologyFromOEMol(ligand_input)
                ligand_md_topology = md.Topology.from_openmm(ligand_topology)
                ligand_n_atoms = ligand_md_topology.n_atoms

            else:
                _logger.warning(f'ligand filetype not recognised. Please provide a path to a .pdb or .sdf file')
                return

            if solvate:
                # Now create a complex
                complex_md_topology = protein_md_topology.join(ligand_md_topology)
                complex_topology = complex_md_topology.to_openmm()
                complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
                complex_positions[:protein_n_atoms, :] = protein_positions
                complex_positions[protein_n_atoms:, :] = ligand_positions

                # Convert positions back to openmm vec3 objects
                complex_positions_vec3 = []
                for position in complex_positions:
                    complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
                complex_positions = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)
            else:
                complex_topology = ligand_topology
                complex_positions = ligand_positions

        # Now for a system_generator
        self.system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

        # Solvate apo and complex (if necessary) and generate systems...
        inputs = []
        topology_list = [protein_topology]
        positions_list = [protein_positions]
        box_dimensions_list = [apo_box_dimensions]
        if ligand_input:
            topology_list.append(complex_topology)
            positions_list.append(complex_positions)
            box_dimensions_list.append(complex_box_dimensions)

        for topology, positions, box_dimensions in zip(topology_list, positions_list, box_dimensions_list):
            if solvate:
                solvated_topology, solvated_positions = self._solvate(topology, positions, water_model, phase,
                                                                      ionic_strength, box_dimensions)
            else:
                solvated_topology = topology
                solvated_positions = unit.quantity.Quantity(value=np.array(
                    [list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]),
                                                            unit=unit.nanometers)
            solvated_system = self.system_generator.create_system(solvated_topology)
            print(solvated_system.getDefaultPeriodicBoxVectors())
            print(solvated_system.usesPeriodicBoundaryConditions())
            inputs.append([solvated_topology, solvated_positions, solvated_system])

        geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


        # Run pipeline...
        htfs = []
        for is_complex, (top, pos, sys) in enumerate(inputs):
            # Change the name of the old residue to its nonstandard name, if necessary
            # Note this needs to happen after generating the system, as the system generator requires standard residue names
            if old_residue:
                for residue in top.residues():
                    if residue.id == mutation_residue_id:
                        residue.name = old_residue
                        print(f"Changed resid {mutation_residue_id} to {residue.name}")

            point_mutation_engine = PointMutationEngine(wildtype_topology=top,
                                                                 system_generator=self.system_generator,
                                                                 chain_id=mutation_chain_id, # Denote the chain id allowed to mutate (it's always a string variable)
                                                                 max_point_mutants=1,
                                                                 residues_allowed_to_mutate=[mutation_residue_id], # The residue ids allowed to mutate
                                                                 allowed_mutations=[(mutation_residue_id, proposed_residue)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                                                 aggregate=True) # Always allow aggregation

            topology_proposal = point_mutation_engine.propose(sys, top, extra_sidechain_map=extra_sidechain_map, demap_CBs=demap_CBs)
            print(topology_proposal.old_system.getDefaultPeriodicBoxVectors())
            print(topology_proposal.old_system.usesPeriodicBoundaryConditions())
            
            # Fix naked charges in old and new systems
            old_topology_atom_map = {atom.index: atom.residue.name for atom in topology_proposal.old_topology.atoms()}
            new_topology_atom_map = {atom.index: atom.residue.name for atom in topology_proposal.new_topology.atoms()}
            for i, system in enumerate([topology_proposal.old_system, topology_proposal.new_system]):
                force_dict = {i.__class__.__name__: i for i in system.getForces()}
                atom_map = old_topology_atom_map if i == 0 else new_topology_atom_map
                if 'NonbondedForce' in [k for k in force_dict.keys()]:
                    nb_force = force_dict['NonbondedForce']
                    for idx in range(nb_force.getNumParticles()):
                        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
                            continue
                        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
                        if sigma == 0*unit.nanometer:
                            new_sigma = 0.06*unit.nanometer
                            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                            _logger.info(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
                        if epsilon == 0*unit.kilojoule_per_mole:
                            new_epsilon = 0.0001*unit.kilojoule_per_mole
                            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
                            _logger.info(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
                            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                                new_sigma = 0.1*unit.nanometer
                                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                                _logger.info(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

            # Only validate energy bookkeeping if the WT and proposed residues do not involve rings
            old_res = [res for res in top.residues() if res.id == mutation_residue_id][0]
            validate_bool = False if old_res.name in ring_amino_acids or proposed_residue in ring_amino_acids else True
            new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pos, beta,
                                                                   validate_energy_bookkeeping=validate_bool)
            logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, pos, beta,
                                                        validate_energy_bookkeeping=validate_bool)

            # Check for charge change...
            if phase != 'vacuum' and use_counterion:
                charge_diff = point_mutation_engine._get_charge_difference(current_resname=topology_proposal.old_topology.residue_topology.name,
                                                                       new_resname=topology_proposal.new_topology.residue_topology.name)
           
                _logger.info(f"charge diff: {charge_diff}")
                if charge_diff != 0:
                    new_water_indices_to_ionize = point_mutation_engine.get_water_indices(charge_diff, new_positions, topology_proposal.new_topology, radius=0.8)
                    _logger.info(f"new water indices to ionize {new_water_indices_to_ionize}")
                    self._get_ion_and_water_parameters(topology_proposal.old_system, topology_proposal.old_topology)
                    self._transform_waters_into_ions(new_water_indices_to_ionize, topology_proposal.new_system, charge_diff)
                    PointMutationExecutor._modify_atom_classes(new_water_indices_to_ionize, topology_proposal)
            elif phase == 'vacuum' or not use_counterion:
                _logger.info("skipping counterion functionality because phase is vacuum")
                    
            if generate_unmodified_hybrid_topology_factory:
                repartitioned_endstate = None
                self.generate_htf(HybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_radius, w_scale)
            if generate_repartitioned_hybrid_topology_factory:
                for repartitioned_endstate in [0, 1]:
                    self.generate_htf(RepartitionedHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_radius, w_scale)
            if generate_rest_capable_hybrid_topology_factory:
                repartitioned_endstate = None
                if rest_radius is None:
                    _logger.info("Trying to generate a RESTCapableHybridTopologyFactory, but rest_radius was not specified. Using 0.2 nm...")
                    rest_radius = 0.2
                self.generate_htf(RESTCapableHybridTopologyFactory, topology_proposal, pos, new_positions, flatten_exceptions, flatten_torsions, repartitioned_endstate, is_complex, rest_radius, w_scale)
                
            if not topology_proposal.unique_new_atoms:
                assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
                assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
            else:
                added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

            if not topology_proposal.unique_old_atoms:
                assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
                assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
                subtracted_valence_energy = 0.0
            else:
                subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

            # Conduct endstate energy validation
            if conduct_endstate_validation:

                assert not flatten_torsions and not flatten_exceptions, "Cannot conduct endstate validation if flatten_torsions or flatten_exceptions is True"

                if generate_unmodified_hybrid_topology_factory:
                    from perses.tests.utils import validate_endstate_energies
                    htf = self.get_complex_htf() if is_complex else self.get_apo_htf()
                    zero_state_error, one_state_error = validate_endstate_energies(htf._topology_proposal,
                                                                                   htf,
                                                                                   added_valence_energy,
                                                                                   subtracted_valence_energy,
                                                                                   beta=beta,
                                                                                   ENERGY_THRESHOLD=ENERGY_THRESHOLD)

                if generate_repartitioned_hybrid_topology_factory:
                    from perses.tests.utils import validate_endstate_energies
                    htf_0 = self.get_complex_rhtf_0() if is_complex else self.get_apo_rhtf_0()
                    htf_1 = self.get_complex_rhtf_1() if is_complex else self.get_apo_rhtf_1()

                    zero_state_error, _ = validate_endstate_energies(htf_0._topology_proposal,
                                                                     htf_0,
                                                                     added_valence_energy,
                                                                     subtracted_valence_energy,
                                                                     ENERGY_THRESHOLD=ENERGY_THRESHOLD,
                                                                     beta=beta,
                                                                     repartitioned_endstate=0)

                    _, one_state_error = validate_endstate_energies(htf_1._topology_proposal,
                                                                    htf_1,
                                                                    added_valence_energy,
                                                                    subtracted_valence_energy,
                                                                    ENERGY_THRESHOLD=ENERGY_THRESHOLD,
                                                                    beta=beta,
                                                                    repartitioned_endstate=1)

                if generate_rest_capable_hybrid_topology_factory:
                    from perses.tests.utils import validate_endstate_energies_point
                    for endstate in [0, 1]:
                        htf = copy.deepcopy(self.get_complex_rest_htf()) if is_complex else copy.deepcopy(self.get_apo_rest_htf())
                        validate_endstate_energies_point(htf, endstate=endstate, minimize=True)

            else:
                pass
    
    def _solvate(self,
               topology,
               positions,
               water_model,
               phase,
               ionic_strength,
               box_dimensions=None):
        """
        Generate a solvated topology, positions, and system for a given input topology and positions.
        For generating the system, the forcefield files provided in the constructor will be used.
        Parameters
        ----------
        topology : app.Topology
            Topology of the system to solvate
        positions : [n, 3] ndarray of Quantity nm
            the positions of the unsolvated system
        forcefield : SystemGenerator.forcefield
            forcefield file of solvent to add
        water_model : str
            solvent model to use for solvation
        phase : str
            if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
        ionic_strength : float * unit.molar
            the total concentration of ions (both positive and negative) to add using Modeller.
            This does not include ions that are added to neutralize the system.
            Note that only monovalent ions are currently supported.
        Returns
        -------
        solvated_topology : app.Topology
            Topology of the system with added waters
        solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
            Solvated positions
        solvated_system : openmm.System
            The parameterized system, containing a barostat if one was specified.
        """
        modeller = app.Modeller(topology, positions)

        # Now we have to add missing atoms
        if phase != 'vacuum':
            _logger.info(f"solvating at {ionic_strength} using {water_model}")
            if not box_dimensions:
                modeller.addSolvent(self.system_generator.forcefield, model=water_model, padding=0.9 * unit.nanometers, ionicStrength=ionic_strength)
            else:
                modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
        else:
            pass

        solvated_topology = modeller.getTopology()
        if box_dimensions:
            solvated_topology.setUnitCellDimensions(box_dimensions)
        solvated_positions = modeller.getPositions()

        # Canonicalize the solvated positions: turn tuples into np.array
        solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

        return solvated_topology, solvated_positions
            
     

In [5]:
def generate_htf(outdir, residue, mutant, apo_file, use_counterion=True, w_scale=0.3):
    solvent_delivery = PointMutationExecutor2(apo_file,
                        '1',
                        residue,
                        mutant,
                        use_counterion=use_counterion,
                        solvate=False,
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        rest_radius=0.3,
                        w_scale=w_scale,
                        flatten_torsions=False,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=False,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                       )

    # Saving htfs as pickles
    _logger.info("Saving htfs as pickles")
    i = os.path.basename(os.path.dirname(outdir))

    apo_rest_htf = solvent_delivery.get_apo_rest_htf()
    with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
        pickle.dump(apo_rest_htf, f)
        
#     apo_htf = solvent_delivery.get_apo_htf()
#     with open(os.path.join(outdir, f"{i}_apo_vanilla.pickle"), "wb") as f:
#         pickle.dump(apo_htf, f)

    # Render atom map
    atom_map_filename = f'{outdir}/atom_map.png'
    render_protein_residue_atom_mapping(apo_rest_htf._topology_proposal, atom_map_filename)

    # Save pdbs
    app.PDBFile.writeFile(apo_rest_htf._topology_proposal.old_topology, apo_rest_htf.old_positions(apo_rest_htf.hybrid_positions), open(os.path.join(outdir, "apo_old.pdb"), "w"), keepIds=True)
    app.PDBFile.writeFile(apo_rest_htf._topology_proposal.new_topology, apo_rest_htf.new_positions(apo_rest_htf.hybrid_positions), open(os.path.join(outdir, "apo_new.pdb"), "w"), keepIds=True)
    
    # Check the rest htf
    _logger.info(f"sterics expression: {apo_rest_htf.hybrid_system.getForce(5).getEnergyFunction()}")
    _logger.info(f"forces: {apo_rest_htf.hybrid_system.getForces()}")
    _logger.info(f"number of particles in apo: {apo_rest_htf.hybrid_system.getNumParticles()}")
    _logger.info(f"number of particles in apo rest region: {len(apo_rest_htf._rest_region)}")

    print(f"apo rest region atoms:")
    for atom in apo_rest_htf.hybrid_topology.atoms:
        if atom.index in apo_rest_htf._rest_region:
            print(atom)
            
    return solvent_delivery

In [4]:
def generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir):
    i = os.path.basename(os.path.dirname(old_outdir))
    with open(os.path.join(old_outdir, f"{i}_apo.pickle"), "rb") as f:
        htf = pickle.load(f)
    topology_proposal = htf._topology_proposal
    old_positions = htf.old_positions(htf.hybrid_positions)
    new_positions = htf.new_positions(htf.hybrid_positions)
    flatten_exceptions = True
    flatten_torsions = False
    htf = HybridTopologyFactory(topology_proposal=topology_proposal,
                                      current_positions=old_positions,
                                      new_positions=new_positions,
                                      use_dispersion_correction=False,
                                      functions=None,
                                      softcore_alpha=None,
                                      bond_softening_constant=1.0,
                                      angle_softening_constant=1.0,
                                      soften_only_new=False,
                                      neglected_new_angle_terms=[],
                                      neglected_old_angle_terms=[],
                                      softcore_LJ_v2=True,
                                      softcore_electrostatics=True,
                                      softcore_LJ_v2_alpha=0.85,
                                      softcore_electrostatics_alpha=0.3,
                                      softcore_sigma_Q=1.0,
                                      interpolate_old_and_new_14s=flatten_exceptions,
                                      omitted_terms=None,
                                      endstate=None,
                                      flatten_torsions=flatten_torsions)
    
    i = os.path.basename(os.path.dirname(new_outdir))
    with open(os.path.join(new_outdir, f"{i}_apo.pickle"), "wb") as f:
        pickle.dump(htf, f)
        
    # Render atom map
    atom_map_filename = f'{new_outdir}/atom_map.png'
    render_protein_residue_atom_mapping(htf._topology_proposal, atom_map_filename)

    # Save pdbs
    app.PDBFile.writeFile(htf._topology_proposal.old_topology, htf.old_positions(htf.hybrid_positions), open(os.path.join(new_outdir, "apo_old.pdb"), "w"), keepIds=True)
    app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open(os.path.join(new_outdir, "apo_new.pdb"), "w"), keepIds=True)
    
    # Check the rest htf
    _logger.info(f"sterics expression: {htf.hybrid_system.getForce(5).getEnergyFunction()}")
    _logger.info(f"forces: {htf.hybrid_system.getForces()}")
    _logger.info(f"number of particles in apo: {htf.hybrid_system.getNumParticles()}")

    return htf

In [5]:
def generate_vacuum_htf(outdir, residue, mutant, apo_file, w_scale=0.3):
    
    solvent_delivery = PointMutationExecutor2(apo_file,
                        '1',
                        residue,
                        mutant,
                        phase='vacuum',
                        barostat=None,
                        forcefield_files=['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'],
                        periodic_forcefield_kwargs=None,
                        nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                        rest_radius=0.3,
                        w_scale=w_scale,
                        flatten_torsions=False,
                        flatten_exceptions=True,
                        generate_unmodified_hybrid_topology_factory=True,
                        generate_rest_capable_hybrid_topology_factory=True,
                        conduct_endstate_validation=False
                       )

    # Saving htfs as pickles
    _logger.info("Saving htfs as pickles")
    i = os.path.basename(os.path.dirname(outdir))

    apo_rest_htf = solvent_delivery.get_apo_rest_htf()
    with open(os.path.join(outdir, f"{i}_apo.pickle"), "wb") as f:
        pickle.dump(apo_rest_htf, f)
    
    apo_htf = solvent_delivery.get_apo_htf()
    with open(os.path.join(outdir, f"{i}_apo_vanilla.pickle"), "wb") as f:
        pickle.dump(apo_htf, f)

    # Render atom map
    atom_map_filename = f'{outdir}/atom_map.png'
    render_protein_residue_atom_mapping(apo_rest_htf._topology_proposal, atom_map_filename)

    # Save pdbs
    app.PDBFile.writeFile(apo_rest_htf._topology_proposal.old_topology, apo_rest_htf.old_positions(apo_rest_htf.hybrid_positions), open(os.path.join(outdir, "apo_old.pdb"), "w"), keepIds=True)
    app.PDBFile.writeFile(apo_rest_htf._topology_proposal.new_topology, apo_rest_htf.new_positions(apo_rest_htf.hybrid_positions), open(os.path.join(outdir, "apo_new.pdb"), "w"), keepIds=True)
    
    # Check the rest htf
    _logger.info(f"sterics expression: {apo_rest_htf.hybrid_system.getForce(5).getEnergyFunction()}")
    _logger.info(f"forces: {apo_rest_htf.hybrid_system.getForces()}")
    _logger.info(f"number of particles in apo: {apo_rest_htf.hybrid_system.getNumParticles()}")
    _logger.info(f"number of particles in apo rest region: {len(apo_rest_htf._rest_region)}")

    print(f"apo rest region atoms:")
    for atom in apo_rest_htf.hybrid_topology.atoms:
        if atom.index in apo_rest_htf._rest_region:
            print(atom)
            
    return solvent_delivery

    

In [6]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/0/"
residue = '2'
mutant = 'ASP'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/0/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...


(Vec3(x=2.9625000000000004, y=0.0, z=0.0), Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337)) nm
[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 5
INFO:geometry:Atom index proposal order is [15, 16, 17, 11, 12]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 12 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 39 angles in reference force.
INFO:geometry:	creating torsion force...
INF

[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


KeyboardInterrupt: 

In [10]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/1/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/1/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.8033, y=0.0, z=0.0), Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0), Vec3(x=-0.9344323814922079, y=1.3214864422375598, z=2.2888863225160656)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...


[Quantity(value=Vec3(x=2.8033, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9344323814922079, y=1.3214864422375598, z=2.2888863225160656), unit=nanometer)]
True


INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 27, 24, 17, 20, 22, 19, 12, 11, 28, 16, 29, 26, 25]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 15 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 60 angles in reference fo

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 41.41758474299305
INFO:geometry:final reduced energy 82.97836882897181
INFO:geometry:sum of energies: 82.97837354114402
INFO:geometry:magnitude of difference in the energies: 4.71217221331699e-06
INFO:geometry:Final logp_proposal: -0.7964708151372104


added energy components: [('CustomBondForce', 0.37716629845405375), ('CustomAngleForce', 4.7222212616231705), ('CustomTorsionForce', 11.049402852182757), ('CustomBondForce', 25.26879433073308)]


INFO:setup:charge diff: -2
INFO:setup:new water indices to ionize [ 54  55  56 375 376 377]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.8033, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0)

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ASP2-N
ASP2-H
ASP2-CA
ASP2-HA
ASP2-CB
ASP2-HB2
ASP2-HB3
ASP2-C
ASP2-O
ASP2-CG
ASP2-OD1
ASP2-OD2
ASP2-HB2
ASP2-HB3
ASP2-CG
ASP2-HG2
ASP2-HG3
ASP2-CD
ASP2-HD2
ASP2-HD3
ASP2-NE
ASP2-HE
ASP2-CZ
ASP2-NH1
ASP2-HH11
ASP2-HH12
ASP2-NH2
ASP2-HH21
ASP2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [4]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/2/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/2/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

(Vec3(x=2.9625000000000004, y=0.0, z=0.0), Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337)) nm


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [11, 12, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -478.8342662842031
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946999333319
INFO:geometry:magnitude of difference in the energies: 7.767607996811421e-06
INFO:geometry:Final logp_proposal: -31466.64864041896


added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707969376224987), ('CustomBondForce', -500.77732896921924)]


INFO:setup:charge diff: 1
INFO:setup:new water indices to ionize [1873 1874 1875]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [5]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/3/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/3/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm
[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 26, 11, 17, 20, 19, 25, 12, 16, 28, 22, 29]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 15 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 60 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 78 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference no

[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (28, 27, 23, 21, [1, Quantity(value=3.1400479780546657, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 15]), 1: (28, 27, 23, 24, [1, Quantity(value=-0.0009854700401916716, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 15]), 2: (29, 27, 23, 21, [1, Quantity(value=-0.0019499207113558015, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 17]), 3: (29, 27, 23, 24, [1, Quantity(value=3.1402019383733304, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 17]), 4: (27, 23, 24, 26, [1, Quantity(value=-3.141011085169936, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 7]), 5: (27, 23, 24, 25, [1, Quantity(value=0.0013841937088012735, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 12]), 6: (21, 23, 24, 26, [1, Quantity(value=0.0011725432810614222, unit=rad

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.37413019278795
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.73453742825751
INFO:geometry:magnitude of difference in the energies: 5.335226429536988e-07
INFO:geometry:Final logp_proposal: -29370.926273094494


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379330968913627), ('CustomBondForce', 8.21577368679069)]


INFO:setup:charge diff: -1
INFO:setup:new water indices to ionize [843 844 845]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nano

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [7]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/4/"
residue = '2'
mutant = 'THR'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/4/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 13, 12: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm
[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:setup:Changed particle 19's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 19's sigma from 1.0 nm to 0.1 nm
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [14, 18, 19, 15, 16, 13, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1648 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (19, 18, 10, 8, [1, Quantity(value=-0.07

...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 31.36040723546956


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29547.7853150218
INFO:setup:charge diff: 0
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to h

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB
ALA2-CG2
ALA2-HG21
ALA2-HG22
ALA2-HG23
ALA2-OG1
ALA2-HG1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [9]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/5/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/5/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 13, 12: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('CA', 'CA'), ('C', 'C'), ('O', 'O'), ('CB', 'CB'), ('H', 'H'), ('HA', 'HA')]
INFO:setup:Changed particle 16's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 16's sigma from 1.0 nm to 0.1 nm
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [14, 15, 13]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old sy

[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True
[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1631 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (6, 7, 10, 13, [1, Quantity(value=2.10535

conducting subsequent work with the following platform: CUDA


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -49.39389917948375
INFO:geometry:final reduced energy -11.906030570911103
INFO:geometry:sum of energies: -11.906031567278468
INFO:geometry:magnitude of difference in the energies: 9.96367361949524e-07
INFO:geometry:Final logp_proposal: -5878.947678171476
INFO:setup:charge diff: 0
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***


added energy components: [('CustomBondForce', 2.491618312600117), ('CustomAngleForce', 10.012960961555445), ('CustomTorsionForce', 8.024630559229106), ('CustomBondForce', -69.92310901286842)]


INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
INFO:relative:Determined atom clas

apo rest region atoms:
ACE1-C
ACE1-O
ACE1-CH3
ACE1-H1
ACE1-H2
ACE1-H3
THR2-N
THR2-CA
THR2-C
THR2-O
THR2-CB
THR2-CG2
THR2-OG1
THR2-H
THR2-HA
THR2-HB
THR2-HG1
THR2-HG21
THR2-HG22
THR2-HG23
THR2-HB1
THR2-HB2
THR2-HB3
NME3-N
NME3-C
NME3-H
NME3-H1
NME3-H2
NME3-H3


In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/6/"
residue = '2'
mutant = 'VAL'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/6/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 13, 12: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 9
INFO:geometry:Atom index proposal order is [14, 18, 20, 17, 15, 13, 19, 21, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 31.36040723546956


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29567.071580180043
INFO:setup:charge diff: 0
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB
ALA2-CG1
ALA2-HG11
ALA2-HG12
ALA2-HG13
ALA2-CG2
ALA2-HG21
ALA2-HG22
ALA2-HG23
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [10]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/7/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/7/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [14, 15, 13]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True
[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1625 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (6, 8, 10, 13, [1, Quantity(value=2.10535

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -43.5245826250011
INFO:geometry:final reduced energy -5.265391459546188
INFO:geometry:sum of energies: -5.265389472931922
INFO:geometry:magnitude of difference in the energies: 1.986614265092612e-06
INFO:geometry:Final logp_proposal: -4809.425521748758
INFO:setup:charge diff: 0
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***


added energy components: [('CustomBondForce', 0.7739761929470015), ('CustomAngleForce', 5.851901320036396), ('CustomTorsionForce', 5.638422801555334), ('CustomBondForce', -55.78888293953983)]


INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
INFO:relative:Determined atom clas

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
VAL2-N
VAL2-H
VAL2-CA
VAL2-HA
VAL2-CB
VAL2-C
VAL2-O
VAL2-HB
VAL2-CG1
VAL2-HG11
VAL2-HG12
VAL2-HG13
VAL2-CG2
VAL2-HG21
VAL2-HG22
VAL2-HG23
VAL2-HB1
VAL2-HB2
VAL2-HB3
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [13]:
# check that the hmass is 3 amu
htf = solvent_delivery.get_apo_rest_htf()
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom, htf.hybrid_system.getParticleMass(atom.index))

0 ACE1-H1 3.0 Da
1 ACE1-CH3 6.034 Da
2 ACE1-H2 3.0 Da
3 ACE1-H3 3.0 Da
4 ACE1-C 12.01 Da
5 ACE1-O 16.0 Da
6 VAL2-N 12.018 Da
7 VAL2-H 3.0 Da
8 VAL2-CA 10.018 Da
9 VAL2-HA 3.0 Da
10 VAL2-CB 8.026 Da
11 VAL2-C 12.01 Da
12 VAL2-O 16.0 Da
13 VAL2-HB 3.0 Da
14 VAL2-CG1 6.034 Da
15 VAL2-HG11 3.0 Da
16 VAL2-HG12 3.0 Da
17 VAL2-HG13 3.0 Da
18 VAL2-CG2 6.034 Da
19 VAL2-HG21 3.0 Da
20 VAL2-HG22 3.0 Da
21 VAL2-HG23 3.0 Da
1557 VAL2-HB1 3.0 Da
1558 VAL2-HB2 3.0 Da
1559 VAL2-HB3 3.0 Da
22 NME3-N 12.018 Da
23 NME3-H 3.0 Da
24 NME3-C 6.034 Da
25 NME3-H1 3.0 Da
26 NME3-H2 3.0 Da
27 NME3-H3 3.0 Da
28 HOH4-O 15.99943 Da
29 HOH4-H1 1.007947 Da
30 HOH4-H2 1.007947 Da
31 HOH5-O 15.99943 Da
32 HOH5-H1 1.007947 Da
33 HOH5-H2 1.007947 Da
34 HOH6-O 15.99943 Da
35 HOH6-H1 1.007947 Da
36 HOH6-H2 1.007947 Da
37 HOH7-O 15.99943 Da
38 HOH7-H1 1.007947 Da
39 HOH7-H2 1.007947 Da
40 HOH8-O 15.99943 Da
41 HOH8-H1 1.007947 Da
42 HOH8-H2 1.007947 Da
43 HOH9-O 15.99943 Da
44 HOH9-H1 1.007947 Da
45 HOH9-H2 1.007947 Da
46 H

914 HOH300-H1 1.007947 Da
915 HOH300-H2 1.007947 Da
916 HOH301-O 15.99943 Da
917 HOH301-H1 1.007947 Da
918 HOH301-H2 1.007947 Da
919 HOH302-O 15.99943 Da
920 HOH302-H1 1.007947 Da
921 HOH302-H2 1.007947 Da
922 HOH303-O 15.99943 Da
923 HOH303-H1 1.007947 Da
924 HOH303-H2 1.007947 Da
925 HOH304-O 15.99943 Da
926 HOH304-H1 1.007947 Da
927 HOH304-H2 1.007947 Da
928 HOH305-O 15.99943 Da
929 HOH305-H1 1.007947 Da
930 HOH305-H2 1.007947 Da
931 HOH306-O 15.99943 Da
932 HOH306-H1 1.007947 Da
933 HOH306-H2 1.007947 Da
934 HOH307-O 15.99943 Da
935 HOH307-H1 1.007947 Da
936 HOH307-H2 1.007947 Da
937 HOH308-O 15.99943 Da
938 HOH308-H1 1.007947 Da
939 HOH308-H2 1.007947 Da
940 HOH309-O 15.99943 Da
941 HOH309-H1 1.007947 Da
942 HOH309-H2 1.007947 Da
943 HOH310-O 15.99943 Da
944 HOH310-H1 1.007947 Da
945 HOH310-H2 1.007947 Da
946 HOH311-O 15.99943 Da
947 HOH311-H1 1.007947 Da
948 HOH311-H2 1.007947 Da
949 HOH312-O 15.99943 Da
950 HOH312-H1 1.007947 Da
951 HOH312-H2 1.007947 Da
952 HOH313-O 15.99943 Da

In [29]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/2/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/8/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=

In [30]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/3/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/9/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=

In [31]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/4/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/10/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=

In [32]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/5/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/11/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=

In [34]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/6/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/12/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=

In [35]:
old_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/7/"
new_outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/13/"

htf = generate_vanilla_htf_from_rest_htf(old_outdir, new_outdir)


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=

In [40]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/14/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/14/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/14/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [12, 11, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in refe

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 3 new atoms
INFO:geometry:	reduced angle potential = 0.10100672956376845.
INFO:geometry:	reduced angle potential = 0.04657192575381088.
INFO:geometry:	reduced angle potential = 0.8027319748081378.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced energy added from growth system: -478.8342646200793
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946982692081
INFO:geometry:magnitude of difference in the energies: 9.43173182577084e-06
INFO:geometry:Final logp_proposal: -31453.711521803023


added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707971040348824), ('CustomBondForce', -500.77732896921924)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanom

INFO:setup:number of particles in apo: 39
INFO:setup:number of particles in apo rest region: 39


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [41]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/15/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/15/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/15/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 17, 29, 19, 12, 26, 22, 20, 16, 11, 25, 28]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creati

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 17 new atoms
INFO:geometry:	reduced angle potential = 0.24432277806184183.
INFO:geometry:	reduced angle potential = 0.3509104993980562.
INFO:geometry:	reduced angle potential = 0.12481705738333011.
INFO:geometry:	reduced angle potential = 0.00017170518688130828.
INFO:geometry:	reduced angle potential = 2.0365907309158504.
INFO:geometry:	reduced angle potential = 0.46322282609985344.
INFO:geometry:	reduced angle potential = 2.212069678013874.
INFO:geometry:	reduced angle potential = 0.025464348805780787.
INFO:geometry:	reduced angle potential = 0.4246128970084918.
INFO:geometry:	reduced angle potential = 0.19535869741622236.
INFO:geometry:	reduced angle potential = 0.005424287057707295.
INFO:geometry:	reduced angle potential = 0.1802539285430822.
INFO:geometry:	reduced angle potential = 0.8377704084747088.
INFO:geometry:	reduced angle potential = 4.698272580423476.
INFO:geometry:	reduced angle potential = 0.025255565157004962.
INFO:geometry:	reduced angle potenti

...

INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29441.186879702396


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanom

INFO:setup:number of particles in apo: 39
INFO:setup:number of particles in apo rest region: 39


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [31]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/16/"
residue = '2'
mutant = 'THR'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/16/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/16/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 13, 12: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:setup:Changed particle 19's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
INFO:setup:Changed particle 19's sigma from 1.0 nm to 0.1 nm
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [18, 14, 13, 15, 16, 19, 17]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new posit

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 7 new atoms
INFO:geometry:	reduced angle potential = 1.507268982110531.
INFO:geometry:	reduced angle potential = 0.0006119170259034194.
INFO:geometry:	reduced angle potential = 0.4589241958556348.
INFO:geometry:	reduced angle potential = 1.4444463090787636.
INFO:geometry:	reduced angle potential = 0.03883086779656012.
INFO:geometry:	reduced angle potential = 0.0026741105091615583.
INFO:geometry:	reduced angle potential = 0.00993538035856815.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there

...

INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29576.260175343043


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=

INFO:setup:number of particles in apo: 29
INFO:setup:number of particles in apo rest region: 29


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB
ALA2-CG2
ALA2-HG21
ALA2-HG22
ALA2-HG23
ALA2-OG1
ALA2-HG1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [50]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/19/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/19/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file, use_counterion=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.9625000000000004, y=0.0, z=0.0), Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [12, 11, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -478.8342646200793
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946982692081
INFO:geometry:magnitude of difference in the energies: 9.43173182577084e-06
INFO:geometry:Final logp_proposal: -31442.78237654639


added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707971040348824), ('CustomBondForce', -500.77732896921924)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanomete

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [51]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/20/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/20/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file, use_counterion=False)

(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 19, 16, 12, 22, 11, 28, 17, 25, 26, 20, 29]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29495.87573997401


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [9]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/25/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/25/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

(Vec3(x=2.8033, y=0.0, z=0.0), Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0), Vec3(x=-0.9344323814922079, y=1.3214864422375598, z=2.2888863225160656)) nm


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.8033, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9344323814922079, y=1.3214864422375598, z=2.2888863225160656), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [11, 15, 12]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.8033, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9344323814922082, y=2.642976922793463, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9344323814922079, y=1.3214864422375598, z=2.2888863225160656), unit=nanometer)]
True


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 41.41758474592582
INFO:geometry:final reduced energy 82.97836882897181
INFO:geometry:sum of energies: 82.97837354407679
INFO:geometry:magnitude of difference in the energies: 4.715104985564267e-06
INFO:geometry:Final logp_proposal: -0.13703205683404285
INFO:setup:charge diff: -1


added energy components: [('CustomBondForce', 0.37716629845405375), ('CustomAngleForce', 4.7222212616231705), ('CustomTorsionForce', 11.049402855115531), ('CustomBondForce', 25.26879433073308)]


INFO:setup:new water indices to ionize [748 749 750]
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.8033, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ASP2-N
ASP2-H
ASP2-CA
ASP2-HA
ASP2-CB
ASP2-HB2
ASP2-HB3
ASP2-C
ASP2-O
ASP2-CG
ASP2-OD1
ASP2-OD2
ASP2-HB2
ASP2-HB3
ASP2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [10]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/26/"
residue = '2'
mutant = 'ASP'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/26/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm
[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 5
INFO:geometry:Atom index proposal order is [15, 17, 16, 11, 12]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 12 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 39 angles in reference force.
INFO:geometry:	creating torsion force...
INF

[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {}
INFO:geometry:neglected angle terms include []
INFO:geometry:log probability choice of torsions and atom order: -5.703782474656201
INFO:geometry:creating platform, integrators, and contexts; setting growth parameter


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.374130239445629
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.734537474915186
INFO:geometry:magnitude of difference in the energies: 5.801803215632617e-07
INFO:geometry:Final logp_proposal: -29345.337091923997


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379335634681493), ('CustomBondForce', 8.21577368679069)]


INFO:setup:charge diff: 1
INFO:setup:new water indices to ionize [672 673 674]
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer)

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-OD1
ALA2-OD2
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/27/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/27/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file, w_scale=1.0)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.9625000000000004, y=0.0, z=0.0), Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874989941036161, y=2.793072141324737, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9874989941036157, y=1.3965339368347203, z=2.4188726609545337), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [11, 15, 12]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -478.8342662842031
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946999333319
INFO:geometry:magnitude of difference in the energies: 7.767607996811421e-06
INFO:geometry:Final logp_proposal: -31445.89040799835


added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707969376224987), ('CustomBondForce', -500.77732896921924)]


INFO:setup:charge diff: 1
INFO:setup:new water indices to ionize [415 416 417]
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), uni

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [9]:
 outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/28/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/28/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file, w_scale=1.0)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156)) nm


INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9215990612312835, y=2.6066787700707614, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9215990612312832, y=1.303337393606965, z=2.2574511841374156), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 26, 11, 22, 16, 17, 20, 19, 25, 28, 12, 29]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.37413019278795
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.73453742825751
INFO:geometry:magnitude of difference in the energies: 5.335226429536988e-07
INFO:geometry:Final logp_proposal: -29402.849642168523


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379330968913627), ('CustomBondForce', 8.21577368679069)]


INFO:setup:charge diff: -1
INFO:setup:new water indices to ionize [981 982 983]
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.7648, y=0.0, z=0.0), unit=nanometer

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [13]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/29/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/29/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/29/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum, w_scale=1.0)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [11, 12, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in refe

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 3 new atoms
INFO:geometry:	reduced angle potential = 0.08976471143527215.
INFO:geometry:	reduced angle potential = 0.7269123231566724.
INFO:geometry:	reduced angle potential = 0.020494395439606742.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced energy added from growth system: -478.8342662842031
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946999333319
INFO:geometry:magnitude of difference in the energies: 7.767607996811421e-06
INFO:geometry:Final logp_proposal: -31446.19866757957


added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707969376224987), ('CustomBondForce', -500.77732896921924)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanom

INFO:setup:number of particles in apo: 39
INFO:setup:number of particles in apo rest region: 39


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [14]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/30/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/30/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/30/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum, w_scale=1.0)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 11, 22, 20, 12, 25, 26, 16, 19, 29, 17, 28]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 15 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 60 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 78 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference no

[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

INFO:geometry:total reduced energy added from growth system: 10.37413019278795
INFO:geometry:final reduced energy 41.734536894734866
INFO:geometry:sum of energies: 41.73453742825751
INFO:geometry:magnitude of difference in the energies: 5.335226429536988e-07
INFO:geometry:Final logp_proposal: -29376.86925210625


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.844563196308122), ('CustomTorsionForce', 0.31379330968913627), ('CustomBondForce', 8.21577368679069)]


INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanom

INFO:setup:number of particles in apo: 39
INFO:setup:number of particles in apo rest region: 39


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [7]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/31/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/31/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/31/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum, w_scale=0.2)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [15, 11, 12]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in refe

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False


INFO:geometry:creating platform, integrators, and contexts; setting growth parameter


conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -478.8342646200793
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946982692081
INFO:geometry:magnitude of difference in the energies: 9.43173182577084e-06
INFO:geometry:Final logp_proposal: -31465.500807579832
INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and m

added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707971040348824), ('CustomBondForce', -500.77732896921924)]


INFO:relative:Handling torsion forces...
INFO:relative:	handle_periodic_torsion_forces: looping through old_system to add relevant terms...
INFO:relative:	handle_periodic_torsion_forces: looping through new_system to add relevant terms...
INFO:relative:Handling nonbonded forces...
INFO:relative:	handle_nonbonded: looping through all particles in hybrid...
INFO:relative:	handle_nonbonded: Handling Interaction Groups...
INFO:relative:	handle_nonbonded: Handling Hybrid Exceptions...
INFO:relative:	handle_nonbonded: Handling Original Exceptions...
INFO:relative:Handling unique_new/old interaction exceptions...
INFO:relative:There are old or new system exceptions...proceeding.
INFO:relative:Imposing virtual bonds to ensure system is imaged together.
INFO:relative:		_impose_virtual_bonds: No restraint added because sets overlap (core_atoms=[32, 1, 4, 5, 6, 8, 10, 13, 14, 30], protein_atoms=[8])
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces:

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/32/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/32/equil/apo_equilibrated.cif" 
cif = app.PDBxFile(apo_file)
modeller = app.Modeller(cif.topology, cif.positions)
solvent_chains = [chain for chain in modeller.topology.chains() if chain.index > 0]
modeller.delete(solvent_chains)
modeller.topology.setPeriodicBoxVectors(None)
apo_file_vacuum = "/data/chodera/zhangi/perses_benchmark/repex/38/32/equil/apo_equilibrated_vacuum.cif" 
app.PDBxFile.writeFile(modeller.topology, modeller.positions, open(apo_file_vacuum, "w"), keepIds=True)

solvent_delivery = generate_vacuum_htf(outdir, residue, mutant, apo_file_vacuum, w_scale=0.01)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [11, 12, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in refe

None
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
False
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 3 new atoms
INFO:geometry:	reduced angle potential = 0.177408978061704.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.2828471794178054.
INFO:geometry:	reduced angle potential = 0.0003349974895393463.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 42.539566350871226


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 8.159981229252093), ('CustomAngleForce', 9.07511207953895), ('CustomTorsionForce', 4.707971040348824), ('CustomBondForce', -500.77732896921924)]


INFO:geometry:total reduced energy added from growth system: -478.8342646200793
INFO:geometry:final reduced energy -436.2947077009399
INFO:geometry:sum of energies: -436.2946982692081
INFO:geometry:magnitude of difference in the energies: 9.43173182577084e-06
INFO:geometry:Final logp_proposal: -31459.04893226221
INFO:setup:skipping counterion functionality because phase is vacuum
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and ma

INFO:setup:number of particles in apo: 39
INFO:setup:number of particles in apo rest region: 39


apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ARG2-N
ARG2-H
ARG2-CA
ARG2-HA
ARG2-CB
ARG2-HB2
ARG2-HB3
ARG2-C
ARG2-O
ARG2-CG
ARG2-HG2
ARG2-HG3
ARG2-CD
ARG2-HD2
ARG2-HD3
ARG2-NE
ARG2-HE
ARG2-CZ
ARG2-NH1
ARG2-HH11
ARG2-HH12
ARG2-NH2
ARG2-HH21
ARG2-HH22
ARG2-HB2
ARG2-HB3
ARG2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [6]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/2/2_apo.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
1873

1873

In [ ]:
htf._topology_proposal.

In [7]:
new_sysetm htf._topology_proposal.old_system.getForces()

[<openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b62406eb600> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b62406eb510> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b62406eb630> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b62406eb8d0> >,
 <openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b62406eba20> >]

# Use ALA from ARG->ALA (38/2)

In [8]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/34/"
residue = '2'
mutant = 'ARG'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/2/apo_new.pdb" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874404959209033, y=2.793092822842714, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.987440495920903, y=1.396420187060082, z=2.418962210987714), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 17
INFO:geometry:Atom index proposal order is [15, 18, 21, 23, 24, 27, 20, 28, 16, 11, 17, 29, 26, 22, 19, 12, 25]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874404959209033, y=2.793092822842714, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.987440495920903, y=1.396420187060082, z=2.418962210987714), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 15 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 60 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 78 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 2041 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (28, 27, 23, 21, [1, Quantity(value=3.14

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 10.974568849320748
INFO:geometry:final reduced energy 53.5558390902291
INFO:geometry:sum of energies: 53.55583969255785
INFO:geometry:magnitude of difference in the energies: 6.023287557610502e-07
INFO:geometry:Final logp_proposal: 20.969588677627858
INFO:setup:charge diff: -1


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 2.883924121084282), ('CustomTorsionForce', 0.09095590967033956), ('CustomBondForce', 7.999688818566126)]


INFO:setup:new water indices to ionize [1422 1423 1424]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9625000000000004, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9874404959209033, y=2.793092822842714, z=0.0), unit=nanometer), Quant

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-NE
ALA2-HE
ALA2-CZ
ALA2-NH1
ALA2-HH11
ALA2-HH12
ALA2-NH2
ALA2-HH21
ALA2-HH22
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


In [9]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/35/"
residue = '2'
mutant = 'ALA'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/35/equil/apo_equilibrated.cif" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880989934924385, y=2.794769197815668, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9880989934924381, y=1.3973824637836831, z=2.420342355735873), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 3
INFO:geometry:Atom index proposal order is [12, 11, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880989934924385, y=2.794769197815668, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9880989934924381, y=1.3973824637836831, z=2.420342355735873), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 9 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1997 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (6, 8, 10, 15, [1, Quantity(value=2.10535

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 60.688107042530426
INFO:geometry:final reduced energy 99.89384707267
INFO:geometry:sum of energies: 99.89384787478033
INFO:geometry:magnitude of difference in the energies: 8.021103283795128e-07
INFO:geometry:Final logp_proposal: -16155.601309957654
INFO:setup:charge diff: 1


added energy components: [('CustomBondForce', 2.6749765390964386), ('CustomAngleForce', 13.451238768879307), ('CustomTorsionForce', 2.127525526518583), ('CustomBondForce', 42.4343662080361)]


INFO:setup:new water indices to ionize [976 977 978]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880989934924385, y=2.794769197815668, z=0.0), unit=nanometer), Quantity(value=Vec3(

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
LYS2-N
LYS2-H
LYS2-CA
LYS2-HA
LYS2-CB
LYS2-HB2
LYS2-HB3
LYS2-C
LYS2-O
LYS2-CG
LYS2-HG2
LYS2-HG3
LYS2-CD
LYS2-HD2
LYS2-HD3
LYS2-CE
LYS2-HE2
LYS2-HE3
LYS2-NZ
LYS2-HZ1
LYS2-HZ2
LYS2-HZ3
LYS2-HB2
LYS2-HB3
LYS2-HB1
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3


# Use ALA from LYS->ALA (38/35)

In [10]:
outdir = "/data/chodera/zhangi/perses_benchmark/repex/38/36/"
residue = '2'
mutant = 'LYS'

apo_file = "/data/chodera/zhangi/perses_benchmark/repex/38/35/apo_new.pdb" 

solvent_delivery = generate_htf(outdir, residue, mutant, apo_file)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 13: 13, 14: 14}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


[Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880404597665261, y=2.7947898918996312, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9880404597665258, y=1.3972686448952576, z=2.4204319601792), unit=nanometer)]
True


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 15
INFO:geometry:Atom index proposal order is [15, 18, 21, 24, 22, 20, 12, 17, 26, 19, 16, 23, 27, 11, 25]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


[Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880404597665261, y=2.7947898918996312, z=0.0), unit=nanometer), Quantity(value=Vec3(x=-0.9880404597665258, y=1.3972686448952576, z=2.4204319601792), unit=nanometer)]
True


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 13 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 60 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 82 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 2069 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (22, 21, 24, 27, [1, Quantity(value=-2.4

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 9.213144516853129
INFO:geometry:final reduced energy 48.37198639204966
INFO:geometry:sum of energies: 48.3719869729195
INFO:geometry:magnitude of difference in the energies: 5.808698393394707e-07
INFO:geometry:Final logp_proposal: 19.172800363649323
INFO:setup:charge diff: -1


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.2186352924375459), ('CustomTorsionForce', 0.11318969329564774), ('CustomBondForce', 7.881319531119938)]


INFO:setup:new water indices to ionize [1654 1655 1656]
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 1.0 nm
INFO:relative:alpha_ewald is 2.7569734238004693
INFO:relative:w_scale is 0.3
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.9643, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.9880404597665261, y=2.7947898918996312, z=0.0), unit=nanometer), Quantity(value=V

apo rest region atoms:
ACE1-H1
ACE1-CH3
ACE1-H2
ACE1-H3
ACE1-C
ACE1-O
ALA2-N
ALA2-H
ALA2-CA
ALA2-HA
ALA2-CB
ALA2-HB2
ALA2-HB3
ALA2-C
ALA2-O
ALA2-HB1
ALA2-HB2
ALA2-HB3
ALA2-CG
ALA2-HG2
ALA2-HG3
ALA2-CD
ALA2-HD2
ALA2-HD3
ALA2-CE
ALA2-HE2
ALA2-HE3
ALA2-NZ
ALA2-HZ1
ALA2-HZ2
ALA2-HZ3
NME3-N
NME3-H
NME3-C
NME3-H1
NME3-H2
NME3-H3
